In [1]:
from clearml import Task, OutputModel
from cub_tools.trainer import ClearML_Ignite_Trainer
import furl
import pathlib
import tempfile
from datetime import datetime

In [3]:
model_config = '/home/edmorris/projects/image_classification/caltech_birds/scripts/configs/torchvision/resnet34_config.yaml'

In [4]:
task = Task.get_task(task_id="cedea43ac417467387d9d411cb794848")

In [5]:
assert len(task.get_models()['output']) == 1,'[ERROR] More than one model detected, unable to proceed.'

In [6]:
dirname = tempfile.mkdtemp(prefix=f"ignite_torchscripts_{datetime.now().strftime('%Y_%m_%d_%H_%M_%S_')}")
temp_file_path = os.path.join(dirname,'model.pt')

In [7]:
import torch

# Local modules
from cub_tools.trainer import ClearML_Ignite_Trainer
from cub_tools.args import get_parser
from cub_tools.config import get_cfg_defaults, get_key_value_dict

In [8]:
cmd_args = [
    'DIRS.CLEAN_UP', False,     # Don't do anything to the directory structure.
    'MODEL.PRETRAINED', False,  # Don't load default weights, as we want to load our own.
    ]  
trainer = ClearML_Ignite_Trainer(task=task,config=model_config, cmd_args=cmd_args)

[INFO] Parameters Override:: ['DIRS.CLEAN_UP', False, 'MODEL.PRETRAINED', False]
DATA:
  DATA_DIR: /home/edmorris/projects/image_classification/caltech_birds/data/images
  NUM_CLASSES: 200
  TEST_DIR: test
  TRAIN_DIR: train
  TRANSFORMS:
    PARAMS:
      AGGRESIVE:
        persp_distortion_scale: 0.25
        rotation_range: (-10.0, 10.0)
        type: all
      DEFAULT:
        img_crop_size: 224
        img_resize: 256
    TYPE: default
DIRS:
  CLEAN_UP: False
  ROOT_DIR: /home/edmorris/projects/image_classification/caltech_birds
  WORKING_DIR: /home/edmorris/projects/image_classification/caltech_birds/models/classification/ignite_resnet34
EARLY_STOPPING_PATIENCE: 5
MODEL:
  MODEL_LIBRARY: torchvision
  MODEL_NAME: resnet34
  PRETRAINED: False
  WITH_AMP: False
  WITH_GRAD_SCALE: False
SYSTEM:
  LOG_HISTORY: True
TRAIN:
  BATCH_SIZE: 16
  LOSS:
    CRITERION: CrossEntropy
  NUM_EPOCHS: 40
  NUM_WORKERS: 4
  OPTIMIZER:
    PARAMS:
      lr: 0.001
      momentum: 0.9
      nesterov: 

In [9]:
trainer.create_model(load_to_device=False)

[INFO] Successfully created model but NOT pushed it to the device cuda:0
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [16, 64, 112, 112]           9,408
       BatchNorm2d-2         [16, 64, 112, 112]             128
              ReLU-3         [16, 64, 112, 112]               0
         MaxPool2d-4           [16, 64, 56, 56]               0
            Conv2d-5           [16, 64, 56, 56]          36,864
       BatchNorm2d-6           [16, 64, 56, 56]             128
              ReLU-7           [16, 64, 56, 56]               0
            Conv2d-8           [16, 64, 56, 56]          36,864
       BatchNorm2d-9           [16, 64, 56, 56]             128
             ReLU-10           [16, 64, 56, 56]               0
       BasicBlock-11           [16, 64, 56, 56]               0
           Conv2d-12           [16, 64, 56, 56]          36,864
      BatchNorm2d-13          

In [10]:
chkpnt_model.

TRAIN [Network: resnet34, Library: torchvision] Ignite Train PyTorch CNN on CUB200: cub200_resnet34_ignite_best_model_5_val_acc=0.6940.pt
azure://clearmllibrary/artefacts/Caltech Birds%2FTraining/TRAIN [Network%3A resnet34, Library%3A torchvision] Ignite Train PyTorch CNN on CUB200.cedea43ac417467387d9d411cb794848/models/cub200_resnet34_ignite_best_model_0.pt
2021-06-14 10:12:47,950 - clearml.storage - INFO - Downloading: 5.00MB / 81.72MB @ 25.58MBs from azure://clearmllibrary/artefacts/Caltech Birds%2FTraining/TRAIN [Network%3A resnet34, Library%3A torchvision] Ignite Train PyTorch CNN on CUB200.cedea43ac417467387d9d411cb794848/models/cub200_resnet34_ignite_best_model_0.pt
2021-06-14 10:12:48,182 - clearml.storage - INFO - Downloading: 13.00MB / 81.72MB @ 34.53MBs from azure://clearmllibrary/artefacts/Caltech Birds%2FTraining/TRAIN [Network%3A resnet34, Library%3A torchvision] Ignite Train PyTorch CNN on CUB200.cedea43ac417467387d9d411cb794848/models/cub200_resnet34_ignite_best_model_

In [ ]:
# Get the best model weights file for this experiment
for chkpnt_model in trainer.task.get_models()['output']:
    print(chkpnt_model.name)
    print(chkpnt_model.url)
    if "best_model" in chkpnt_model.name:
        break

# Get the model weights file locally and update the model
local_cache_path = chkpnt_model.get_local_copy()
trainer.update_model_from_checkpoint(checkpoint_file=local_cache_path)

In [18]:
chkpnt_model.url

'azure://clearmllibrary/artefacts/Caltech Birds%2FTraining/TRAIN [Network%3A resnet34, Library%3A torchvision] Ignite Train PyTorch CNN on CUB200.cedea43ac417467387d9d411cb794848/models/cub200_resnet34_ignite_best_model_0.pt'

In [11]:
# Get a sample dataset for running inference with
trainer.create_datatransforms()
trainer.create_dataloaders(shuffle={'train' : True, 'test' : True})

***********************************************
**            DATASET SUMMARY                **
***********************************************
train  size::  5994  images
test  size::  5794  images
Number of classes::  200
***********************************************
[INFO] Created data loaders.


In [12]:
# Create an image batch
X, y = next(iter(trainer.val_loader))
# Push the input images to the device
X = X.to(trainer.device)
# Trace the model
traced_module = torch.jit.trace(trainer.model, (X))
# Write the trace module of the model to disk
traced_module.save(temp_file_path) ### TODO: Need to work out where this is saved, and how to push to an artefact.

In [37]:
new_model_furl

In [14]:
# Build the remote location of the torchscript file, based on the best model weights
# Create furl object of existing model weights
model_furl = furl.furl(chkpnt_model.url)
# Strip off the model path
model_path = pathlib.Path(model_furl.pathstr)
# Get the existing model weights name, and split the name from the file extension.
file_split = os.path.splitext(model_path.name)
# Create the torchscript filename
#if fname is None:
fname = file_split[0]+"_torchscript"+file_split[1]
# Construct the new full uri with the new filename
new_model_furl = furl.furl(origin=model_furl.origin, path=os.path.join(model_path.parent,fname))

furl('azure://clearmllibrary/artefacts/Caltech%20Birds%252FTraining/TRAIN%20%5BNetwork%253A%20resnet34,%20Library%253A%20torchvision%5D%20Ignite%20Train%20PyTorch%20CNN%20on%20CUB200.cedea43ac417467387d9d411cb794848/models')

In [38]:
new_model_furl

furl('azure://clearmllibrary/artefacts/Caltech%20Birds%252FTraining/TRAIN%20%5BNetwork%253A%20resnet34,%20Library%253A%20torchvision%5D%20Ignite%20Train%20PyTorch%20CNN%20on%20CUB200.cedea43ac417467387d9d411cb794848/models/cub200_resnet34_ignite_best_model_0_torchscript.pt')

In [ ]:
# Upload the torchscript model file to the clearml-server
new_output_model = OutputModel(task=trainer.task)

In [ ]:
new_output_model.update_weights(
            weights_filename=temp_file_path,
            upload_uri=new_model_furl.url
            )

furl('azure://clearmllibrary/artefacts/Caltech%20Birds%252FTraining/TRAIN%20%5BNetwork%253A%20resnet34,%20Library%253A%20torchvision%5D%20Ignite%20Train%20PyTorch%20CNN%20on%20CUB200.cedea43ac417467387d9d411cb794848/models?cub200_resnet34_ignite_best_model_0_torchscript.pt&%2Fcub200_resnet34_ignite_best_model_0_torchscript.pt')